In [3]:
import numpy as np
import sympy as sp
import math as m
import matplotlib.pyplot as plt

In [134]:
#puntos = puntos a  grafifcar
#f = funcion a graficar 
def grafico(puntos_x,puntos_y,f,tituloGrafico:str):
    PUNTO_X_MAX =  max(puntos_x)
    PUNTO_X_MIN = min(puntos_x)
    DOMINIO = 5
    #genero datos para el dominio de x
    x_values = np.arange(PUNTO_X_MIN-DOMINIO,DOMINIO+PUNTO_X_MAX,step=0.01)
    #genero datos para la imagen de y
    y_values = ([f(x) for x in x_values])
    #ploteo el grafico
    plt.plot(x_values,y_values)
    # Agregar el eje de abscisas
    plt.axhline(y=0, color='black', linestyle='--')
    # Agregar el eje de ordenadas
    plt.axvline(x=0, color='black', linestyle='--')
    #agrego puntos calulados 
    plt.scatter(puntos_x,puntos_y,c="red")
    #agrego labels
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.title(tituloGrafico)
    PUNTOS_Y_MIN = min(puntos_y)
    PUNTOS_Y_MAX = max(puntos_y)
    plt.ylim(PUNTOS_Y_MIN-DOMINIO,PUNTOS_Y_MAX+DOMINIO)
    #muestro el grafico
    plt.show()

In [5]:
def decimal_truncate(numero, decimales):
    if (decimales < 0): raise Exception("el N° de decimales a truncar debe ser positivo")
    factor = 10.0 ** decimales
    return m.trunc(numero * factor) / factor


In [150]:
#funcion que ingresa los decimales exactos
def decExactos_a_tolerancia(exactitud):
    if (exactitud < 0): raise Exception("La exactitud debe ser positiva o 0")
    #scamos el valor de paradad dada la exactitud
    tolerancia = "0."
    #agregamos tantos 0 como exactitud indicada
    for i in range(exactitud): tolerancia += "0"
    #agregamos un uno para indicar que es una fraccion
    tolerancia += "1"
    #convertimos a flotante para poder operar
    return float(tolerancia)
"""
    x: recibe un valor cercano a la raiz que se busca aproximar
    tolerancia: 0 < Numero < 1 que indica el intervalo de corte de la funcion ej: 0.0001 (como minimo3 deciamles exactos)
    f: funcion de tipo numerica
"""
def aproximar_raiz_newton_secante(x,tolerancia,f):
    symbol = sp.symbols("x")
    #instanciamos derivada de la funcion
    f1 = sp.lambdify(symbol,sp.diff(f(symbol)),np)
    #anidamos funcion para poder utilizar variables no locales
    def secante():
        try:
            return x1 - ((f(x1)*(x1-x))/(f(x1)-f(x)))
        except ZeroDivisionError:
            print(f"error: f({x}) = 0, no se puede dividir por 0")
            exit(6)
    #------------------------- fin secante ----------------------------------    
    while(True):    
        try:
            x1 = x - (f(x)/f1(x))
        except ZeroDivisionError:
            print(f"error: f'({x}) = 0, no se puede dividir por 0")
            exit(6)
        if (abs(x-x1) <= tolerancia):
            break
        x = secante()
    return (x1)

In [7]:
def productoria(points, step, txt=False, **kwargs):
    x =  kwargs['x'] if 'x' in kwargs else points[step][0]
    evaluado = 1
    noEvaluado = ""
    for i in range(step):
        noEvaluado += f'*(x-{points[i][0]})'
        evaluado *= (x - points[i][0])
    return evaluado if not txt else noEvaluado   

In [8]:
def alphas(points:list):
    alphaList = [] 
    
    def prevAlphas(p):
        acum = 0
        for i in range(len(alphaList)):
            acum += alphaList[i] if i < 1 else alphaList[i]*productoria(points,i,x=p)
        return acum

    for i,p in enumerate(points):           
        alpha = (points[i][1]-(prevAlphas(p[0]) if i>0 else 0))/productoria(points, i) 
        alphaList.append(alpha)
       
    return alphaList

In [22]:
def nEquation(points, **kwargs):
    pn =  kwargs['pn'] if 'pn' in kwargs else len(points)
    x = sp.symbols('x')
    equations = []
    alphaList = alphas(points)
    equation = f'{alphaList[0]}'

    for i in range(pn):
        if i > 0:
            equation += f'{"+" if alphaList[i] > 0 else ""}{alphaList[i]}{productoria(points,i,True)}'
    return sp.lambdify(x,equation,np)

In [10]:
def validarNum(txt:str, nat=False, **kwargs):
    min =  kwargs['min'] if 'min' in kwargs else None
    while True:
        val = input(txt)  
        try:
            val = float(val)
            if val.is_integer():
                val = int(val)
            if (min and val < min):
                raise Exception(f"Error: El número debe ser mayor de {min}") 
            if (nat and (val <= 0)):
                raise Exception("Error: Se esperaba un número mayor a 0")                
            return val
        except ValueError:
            print("Error: Se esperaba un valor numérico válido.") 
        except Exception as e:
            print(e)             

In [11]:
def choice(txt:str):
    ch = str(input(txt))
    while(ch.lower() != 's' and ch.lower() != 'n'):
        print(f'Se esperaba S/N pero se obtubo {ch}.')
        ch = str(input(txt))
    return True if ch.lower() == 's' else False

In [125]:
class Puntos:
    def __init__(self, manualStep=False):
        self.steps = int(validarNum('Ingrese la cantidad de puntos:',True)) if manualStep else 20
        self.XN = np.array([])
        self.YN = np.array([])
    
    def generar(self):
        for i in range(self.steps):
            self.XN = np.append(self.XN, validarNum(f'Ingrese el valor para X{i}'))
            self.YN = np.append(self.YN, validarNum(f'Ingrese el valor para Y{i}'))

    def obtenerPuntos(self):
        self.generar()
        return [(x,y) for x,y in zip(self.XN, self.YN)]

class PuntosAleatorios(Puntos):
    def __init__(self, X_Equal_Y, manualStep=False):
        Puntos.__init__(self, manualStep)   
        self.x_min = validarNum('Ingrese la cota menor para x:')
        self.x_max = validarNum('Ingrese la cota mayor para x:',min=self.x_min)
        self.y_min = self.x_min
        self.y_max = self.x_max
        if not X_Equal_Y:
            self.y_min = validarNum('Ingrese la cota inferior para y:')
            self.y_max = validarNum('Ingrese la cota superior para y:',min=self.y_min)
        
    def generar(self):
        #Genera arrays con numeros aleatorios y diferentes entre si
        self.YN = np.random.choice(np.round(np.random.uniform(self.y_min, self.y_max, size = self.steps),3), size=self.steps, replace=False)
        self.XN = np.random.choice(np.round(np.random.uniform(self.x_min, self.x_max, size = self.steps),3), size=self.steps, replace=False)

In [109]:
#cantPuntos = cantidad de puntos que se usaron para generar el polinomio
#funcNewton = polinomio de newton numerico
def testeoGrado(cantPuntos:int,FuncNewton):
    GRADO_TEORICO = cantPuntos-1
    x = sp.symbols("x")
    f_simbolic = sp.simplify(FuncNewton(x))
    GRADO_POLINOMIO = sp.degree(f_simbolic)
    if (GRADO_POLINOMIO != GRADO_TEORICO): raise Exception(f"error: el grado del polinomio ({GRADO_POLINOMIO}) debe ser igual al grado teorico ({GRADO_TEORICO})")
    print(f"El grado del polinomio ({GRADO_POLINOMIO}) es igual al grado teorico (cantPuntos-1 = {GRADO_TEORICO}) ")
    

In [144]:
##Polinomio de larange
def lagrange_interpolation(points):
    n = len(points)
    polynomial = []
    for i in range(n):
        term = []
        numerator = []
        denominator = []
        for j in range(n):
            if i != j:
                numerator.append('(x - {})'.format(points[j][0]))
                denominator.append('({} - {})'.format(points[i][0], points[j][0]))
        term.append('*'.join(numerator))
        term.append('/'.join(denominator))
        polynomial.append('*'.join(term))
    polynomial = '+'.join(polynomial)
    polynomial = sp.expand(polynomial)
    x =  sp.symbols("x")
    return sp.lambdify(x,polynomial,"numpy")

In [153]:
def saltosDeLinea(cantSaltos):
    for i in range(cantSaltos): print("\n")
def generacionPuntosAleatorios(cantPuntos):
    puntos_x = np.random.choice(np.round(np.random.uniform(-10, 30, size=cantPuntos),3), size=cantPuntos, replace=False)
    puntos_y = np.random.choice(np.round(np.random.uniform(-20, 40, size=cantPuntos),3), size=cantPuntos, replace=False)
    return [(x,y) for x,y in zip(puntos_x, puntos_y)]

def main():
    try:
        #funcion de pedido de ingreso de datos a usuario
        #lista_puntos = Puntos().obtenerPuntos() if choice("Desea ingresar los puntos manualmente? S/N") else PuntosAleatorios(choice('Limites de x e y iguales? S/N')).obtenerPuntos()
        CANT_PUNTOS = 20
        lista_puntos = generacionPuntosAleatorios(CANT_PUNTOS)
        lista_puntos_asc = sorted(lista_puntos, key=lambda x: x[0])
        lista_puntos_desc = sorted(lista_puntos, key=lambda x: x[0], reverse=True)
    except Exception:
        print('La creación de la lista de puntos falló.')
        
    ##generacion de simbolo para todas las funciones
    symbol = sp.symbols("x")
    #---------------------------------------------------------------------------------
    saltosDeLinea(2)
    print("POLINOMIO CON X ORDENADO DE MENOR A MAYOR = POL1")
    saltosDeLinea(2)
    print("puntos generados aleatoriamente ordenados segun x de menor a mayor:")
    print(lista_puntos_asc)
    ##calulo de polinomio generado por puntos ordenados segun x de menor a mayor
    saltosDeLinea(2)
    poli_newton_asc = nEquation(lista_puntos_asc)
    print("Polinomio generado:")
    simbolic_poli_newton_asc = sp.simplify(poli_newton_asc(symbol))
    print(simbolic_poli_newton_asc)
    ##Testeo de grado del polinomio ordenado por x
    saltosDeLinea(1)
    testeoGrado(cantPuntos=len(lista_puntos_asc),FuncNewton=poli_newton_asc)
    #grafico de la funcion con sus respectivos puntos
    saltosDeLinea(2)
    PUNTOS_X_ASC =[punto[0] for punto in lista_puntos_asc]
    PUNTOS_Y_ASC = [punto[1] for punto in lista_puntos_asc]
    grafico(PUNTOS_X_ASC,PUNTOS_Y_ASC,poli_newton_asc,"Grafico del polinomio generado con x ordenado de < a >")
    saltosDeLinea(2)
    print("Busqueda de una raiz del POL1 con metood de newton-rapson")
    RAIZ = aproximar_raiz_newton_secante(3,0.001,poli_newton_asc)
    print(f"raiz aproximada con el metodo {decimal_truncate(RAIZ,3)}")
    #---------------------------------------------------------------------------------
    saltosDeLinea(2)
    print("POLINOMIO CON X ORDENADO DE MENOR A MAYOR = POL2")
    ##calulo de polinomio generado por puntos ordenados segun x de menor a mayor
    saltosDeLinea(2)
    print("puntos generados aleatoriamente ordenados segun x de mayor a menor:")
    print(lista_puntos_desc)
    saltosDeLinea(2)
    poli_newton_dec = nEquation(lista_puntos_desc)
    print("Polinomio generado:")
    simbolic_poli_newton_dec = sp.simplify(poli_newton_dec(symbol))
    print(simbolic_poli_newton_dec)
    ##Testeo de grado del polinomio ordenado por x
    saltosDeLinea(1)
    testeoGrado(cantPuntos=len(lista_puntos_desc),FuncNewton=poli_newton_dec)
    #grafico de la funcion con sus respectivos puntos
    saltosDeLinea(2)
    PUNTOS_X_ASC =[punto[0] for punto in lista_puntos_desc]
    PUNTOS_Y_ASC = [punto[1] for punto in lista_puntos_desc]
    grafico(PUNTOS_X_ASC,PUNTOS_Y_ASC,poli_newton_dec,"Grafico del polinomio generado con x ordenado de > a <")
    saltosDeLinea(1)
    print("podemos observar que el grado del polinomio encontrado es igual al anterior:")
    print(f"grdPOL1({sp.degree(simbolic_poli_newton_asc)}) = grdPOL2 = {sp.degree(simbolic_poli_newton_dec)}")
    saltosDeLinea(1)
    print(f"para saber si dos polinomios son iguales estos tienen que serlo en grado y coeficiente a coeficiente")
    print(f"------------------------------ Comparacion visual de polinomios ------------------------------")
    print("POL1")
    print(simbolic_poli_newton_asc)
    print("POL2")
    print(simbolic_poli_newton_dec)
    print (f"se puede observar que los polinomios son iguales, pero por razones de diferencias de aproximacion numerica difieren en los ultimos decimales")
   #---------------------------------------------------------------------------------
    saltosDeLinea(2)
    print("POLINOMIO CON LOS PUNTOS DESORDENADOS = POL3")
    ##calulo de polinomio generado por puntos ordenados segun x de menor a mayor
    saltosDeLinea(2)
    print("puntos generados aleatoriamente desordenados:")
    print(lista_puntos)
    saltosDeLinea(2)
    poli_newton_desor = nEquation(lista_puntos)
    print("Polinomio generado:")
    simbolic_poli_newton_desor = sp.simplify(poli_newton_desor(symbol))
    print(simbolic_poli_newton_desor)
    ##Testeo de grado del polinomio ordenado por x
    saltosDeLinea(1)
    testeoGrado(cantPuntos=len(lista_puntos),FuncNewton=poli_newton_desor)
    #grafico de la funcion con sus respectivos puntos
    saltosDeLinea(2)
    PUNTOS_X_ASC =[punto[0] for punto in lista_puntos]
    PUNTOS_Y_ASC = [punto[1] for punto in lista_puntos]
    grafico(PUNTOS_X_ASC,PUNTOS_Y_ASC,poli_newton_desor,"Grafico del polinomio generado con x desordenado")
    saltosDeLinea(1)
    print("podemos observar que el grado del polinomio encontrado es igual al anterior:")
    print(f"grdPOL1({sp.degree(simbolic_poli_newton_asc)}) = grdPOL3 = {sp.degree(simbolic_poli_newton_desor)}")
    saltosDeLinea(1)
    print(f"para saber si dos polinomios son iguales estos tienen que serlo en grado y coeficiente a coeficiente")
    print(f"------------------------------ Comparacion visual de polinomios ------------------------------")
    print("POL1")
    print(simbolic_poli_newton_asc)
    print("POL3")
    print(simbolic_poli_newton_desor)
    print (f"se puede observar que los polinomios son iguales, pero por razones de diferencias de aproximacion numerica difieren en los ultimos decimales")
    #---------------------------------------------------------------------------------
    saltosDeLinea(2)
    print("POLINOMIO INTERPOLADOR DE LARANGE")
    saltosDeLinea(2)
    print("utilizamos mismos puntos que en el metodo de newton:")
    print(lista_puntos_asc)
    saltosDeLinea(2)
    print("Polinomio generado:")
    poli_larange = lagrange_interpolation(lista_puntos_asc)
    simbolic_poli_laragne = sp.sympify(poli_larange(symbol))
    print(simbolic_poli_laragne)
    saltosDeLinea(2)
    print("-------------------------------- Concluciones --------------------------------")
    print("""Se puede ver que el metodo de larange tambien genera un polinomio de grado teorico
    (cantidad de puntos - 1) pero es diferente en coeficientes""")
    print("\n---------------------------------------------------------------------------------------\n")
    print("""El metodo de larange es más fácil de entender y aplicar que el método de Newton, pero puede ser menos 
    eficiente en términos computacionales cuando se tienen muchos puntos""")
    print("\n---------------------------------------------------------------------------------------\n")
    print("""ambos métodos son útiles para aproximar una función a través de un polinomio interpolador. 
    La elección del método dependerá del número de puntos disponibles y la eficiencia computacional requerida.""")
   
        